# Cross Validation

In [1]:
import sys

sys.path.append("..")
sys.path.append("../../inputs")

In [2]:
import logging

logging.basicConfig(
    # filename=__file__.replace('.py', '.log'),
    level=logging.getLevelName("INFO"),
    format="%(asctime)s [%(levelname)s] [%(module)s] %(message)s",
)

log = logging.getLogger(__name__)

In [3]:
from omegaconf import OmegaConf

c = OmegaConf.load("../config/main.yaml")

c.settings.debug = False
c.wandb.enabled = False
c.wandb.group = "LB"
c.wandb.dir = "../../cache/"
c.settings.dirs.working = ".."
c.settings.dirs.input = "../../inputs/"

pretraind_dir = "../../datasets/trainings"

pretrained = f"""
- dir: {pretraind_dir}/2022-02-01_20-23-49/fold0/
  model: ump_1
- dir: {pretraind_dir}/2022-02-01_20-23-51/fold1/
  model: ump_1
- dir: {pretraind_dir}/2022-02-01_20-23-53/fold2/
  model: ump_1
- dir: {pretraind_dir}/2022-02-01_20-23-55/fold3/
  model: ump_1
- dir: {pretraind_dir}/2022-02-01_20-23-58/fold4/
  model: ump_1
- dir: {pretraind_dir}/2022-02-01_20-24-00/fold5/
  model: ump_1
- dir: {pretraind_dir}/2022-02-01_20-24-02/fold6/
  model: ump_1
- dir: {pretraind_dir}/2022-02-01_20-24-04/fold7/
  model: ump_1
- dir: {pretraind_dir}/2022-02-02_00-43-02/fold8/
  model: ump_1
- dir: {pretraind_dir}/2022-02-02_00-43-04/fold9/
  model: ump_1
- dir: {pretraind_dir}/2022-02-02_00-43-06/fold10/
  model: ump_1
- dir: {pretraind_dir}/2022-02-02_00-43-08/fold11/
  model: ump_1
- dir: {pretraind_dir}/2022-02-02_00-43-11/fold12/
  model: ump_1
- dir: {pretraind_dir}/2022-02-02_00-43-13/fold13/
  model: ump_1
- dir: {pretraind_dir}/2022-02-02_00-43-15/fold14/
  model: ump_1
"""

_pretrained = f"""
- dir: {pretraind_dir}/2022-01-26_07-47-01/fold0/
  model: ump_1
- dir: {pretraind_dir}/2022-01-26_07-47-03/fold1/
  model: ump_1
- dir: {pretraind_dir}/2022-01-26_07-47-05/fold2/
  model: ump_1
- dir: {pretraind_dir}/2022-01-26_07-47-07/fold3/
  model: ump_1
- dir: {pretraind_dir}/2022-01-26_07-47-10/fold4/
  model: ump_1
- dir: {pretraind_dir}/2022-01-26_07-47-12/fold5/
  model: ump_1
- dir: {pretraind_dir}/2022-01-26_07-47-14/fold6/
  model: ump_1

- dir: {pretraind_dir}/2022-01-30_22-12-14/fold0/
  model: ump_1
- dir: {pretraind_dir}/2022-01-30_22-12-16/fold1/
  model: ump_1
- dir: {pretraind_dir}/2022-01-30_22-12-19/fold2/
  model: ump_1
- dir: {pretraind_dir}/2022-01-30_22-12-21/fold3/
  model: ump_1
- dir: {pretraind_dir}/2022-01-30_22-12-23/fold4/
  model: ump_1
"""

c.params.pretrained = OmegaConf.create(pretrained)

log.info(OmegaConf.to_yaml(c))

2022-02-02 08:39:31,866 [INFO] [2515656208] defaults:
- _self_
hydra:
  run:
    dir: ../outputs/${now:%Y-%m-%d_%H-%M-%S}
  job_logging:
    formatters:
      simple:
        format: '%(asctime)s [%(levelname)s][%(module)s] %(message)s'
wandb:
  enabled: true
  entity: imokuri
  project: ump
  dir: ../../cache/
  group: LB
settings:
  print_freq: 100
  gpus: 6,7
  dirs:
    working: ..
    input: ../../inputs/
  inputs:
  - train.csv
  - example_test.csv
  - example_sample_submission.csv
  debug: false
  n_debug_data: 100000
  amp: true
  multi_gpu: true
params:
  seed: 440
  n_class: 1
  n_fold: 5
  skip_training: false
  epoch: 20
  es_patience: 0
  batch_size: 640
  gradient_acc_step: 1
  max_grad_norm: 1000
  fold: simple_cpcv
  group_name: investment_id
  time_name: time_id
  label_name: target
  feature_set:
  - f000
  dataset: ump_1
  model: ump_1
  pretrained:
  - dir: ../../datasets/trainings/2022-02-01_20-23-49/fold0/
    model: ump_1
  - dir: ../../datasets/trainings/2022-02

In [4]:
import os

import pandas as pd
import src.utils as utils
from src.get_score import record_result

In [5]:
run = utils.setup_wandb(c)

2022-02-02 08:39:33,498 [ERROR] [jupyter] Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: imokuri (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.10 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


2022-02-02 08:39:36,984 [INFO] [utils] WandB initialized. name: lucky-ox-78, id: 2baeyx1q


In [6]:
train = pd.read_feather("../../inputs/train.f")
train = train.loc[:, ["row_id", "target"]]
train.set_index("row_id", inplace=True)

In [7]:
preds_col = []

# 各 OOF の結果を読み取り
for n, training in enumerate(c.params.pretrained):
    preds_col.append(f"preds{n}")
    oof_df = pd.read_feather(os.path.join(training.dir.rsplit("/", 2)[0], "oof_df.f")).set_index("row_id")

    train[f"preds{n}"] = oof_df["preds"]

# 各行の OOF の結果の数をカウントする
train["count_oof"] = len(c.params.pretrained) - train.isnull().sum(axis=1)

# OOF の結果がない行を 0 埋め
train.fillna(0, inplace=True)

# OOF の結果をマージ
train["preds"] = 0
for col in preds_col:
    train["preds"] += train[col]

# 推論結果がないものは除外
train = train[train["preds"] != 0.0]

# 複数 OOF を加算している場合は、OOFの数で割る
train["preds"] = train["preds"] / train["count_oof"]

In [8]:
train

,target,preds0,preds1,preds2,preds3,preds4,preds5,preds6,preds7,preds8,preds9,preds10,preds11,preds12,preds13,preds14,count_oof,preds
row_id,,,,,,,,,,,,,,,,,,
0_1,-0.300875,0.086914,0.148071,0.126465,0.085754,0.060791,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,5,0.101599
0_2,-0.231040,-0.026031,0.000782,-0.011414,-0.036316,-0.036072,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,5,-0.021810
0_6,0.568807,0.061188,0.118774,0.101624,0.077271,0.022461,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,5,0.076263
0_7,-1.064780,-0.040497,-0.025497,-0.046356,-0.087952,-0.143311,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,5,-0.068723
0_8,-0.531940,0.014252,0.027084,0.070129,0.102966,0.011040,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,5,0.045094
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1219_3768,0.033600,0.000000,0.000000,0.000000,0.000000,-0.045837,0.0,0.0,0.0,-0.047180,0.0,0.0,-0.065125,0.0,-0.038483,-0.047699,5,-0.048865
1219_3769,-0.223264,0.000000,0.000000,0.000000,0.000000,-0.047729,0.0,0.0,0.0,-0.058228,0.0,0.0,-0.071167,0.0,-0.052917,-0.026688,5,-0.051346
1219_3770,-0.559415,0.000000,0.000000,0.000000,0.000000,0.068298,0.0,0.0,0.0,0.034485,0.0,0.0,0.073364,0.0,0.083984,0.035339,5,0.059094


In [9]:
train["count_oof"].value_counts()

5    3141390
Name: count_oof, dtype: int64

In [10]:
record_result(c, train, c.params.n_fold)

2022-02-02 08:39:59,387 [INFO] [get_score] Score: 0.12271


0.12271253727710651